In [1]:
import requests
import pandas as pd

# Login

In [2]:
# roda acesso, api/
url = "http://127.0.0.1:9000/api/".replace("\\","/")

In [3]:
username="testeuser@zeladorx.com.br"
password="TESte2026&"

body = {
    "username":username,
    "password":password
}

In [4]:
response = requests.post("http://127.0.0.1:9000/api/login", body)

In [5]:
# dados do usuário logado
response.json()

# {
#     'token': '171db50f8cc06dae8ae7338bdc5048ddc97f6784', #token de acesso
#     'id_random': '6uW6BnjPFHyd', #id randomico unico
#     'user_id': 1, #id de chave primaria
#     'username': 'zeladorx', #username
#     'email': 'zeladorx@zeladorx.com.br', #email
#     'is_superuser': True, #é superusario do ecossitema contratado
# }

{'token': '92ca37bb8f53bb9a8fbaefad1155c6ab55677c11',
 'id_random': '6uW6BnjPFHyd',
 'user_id': 1,
 'username': 'zeladorx',
 'email': 'zeladorx@zeladorx.com.br',
 'is_superuser': True}

# Auth

In [6]:
# todas as rotas precisam do tokem para ser acessada
headers = {
    "Authorization": f"Token {response.json()['token']}"
}

# Checklists

In [7]:
# retorna todas os checklists ja criados
# codigo de permissão necessário --> 402: Pode visualizar checklists
response = requests.get("http://127.0.0.1:9000/api/checklist/ListCheckListLimpezaPredial/", headers=headers)
response.json()['results']
df = pd.DataFrame(response.json()['results'])
display(df)

,id,id_random,servico_agendado,descricao,foto_comprovacao,status
0,1,Vqif9zIpzUqA,1,sub teste,None,Pendente


## Criar novo checklist

In [8]:
## As colunas servico_agendado são chaves estrangeiras e precisam estar devidamente configuradas
## sendo obriagatorios

In [8]:
# Lista de empresas que podem ser enviadas pelo usuario
# nao precisa de permissao
print("Serviços agendados")
response = requests.get("http://127.0.0.1:9000/api/servicos/agendados/ListServicoLimpezaPredialmAgendadoAnotados/", headers=headers)
response.json()
df = pd.DataFrame(response.json()['results'])
display(df)
print()

Serviços agendados


,id,id_random,DescricaoDoServico,DataDeInicio,DataDeConclusao,TipoServico,Areas,ServicosEscalados,dias_diferenca,novo_status
0,1,UWoNKAzEeZRs,steste,2025-07-10T18:04:00-03:00,2025-07-10T18:04:00-03:00,Regular,1,[1],-38,Atrasado


In [9]:
body = {
    'servico_agendado': 1, #vem de api/servicos/agendados/ListServicoLimpezaPredialAgendadoAnotados/
    'descricao': "descricao de teste", # descircao do checklist 
    'foto_comprovacao': "", #tipo file comporvando o checklist 
    'status': "Concluído", # opcoes Concluído ou Pendente
}

In [10]:
# codigo de permissão necessário --> 400: Pode criar novos checklists
# essa rota rece o id random do servico que sera crado o checklist, deve ser validado jnto com o id
response = requests.post("http://127.0.0.1:9000/api/checklist/CreateCheckListLimpezaPredial/UWoNKAzEeZRs/", body, headers=headers)
response.json()

{'success': True,
 'message': 'CheckListLimpezaPredial criado com sucesso.',
 'data': {'id': 2,
  'id_random': 'x0GPde9yaCde',
  'servico_agendado': 1,
  'descricao': 'descricao de teste',
  'foto_comprovacao': None,
  'status': 'Concluído'}}

# Detalhes servico checklist

In [15]:
# essa rota recebe o id random da servico como parametro de url, consultar em api/catalogo_de_servicos/ListCheckListLimpezaPredial/
# detalhes do checklist criado anteriormente, obeserve o id_random
# http://127.0.0.1:9000/api/checklist/CheckListLimpezaPredialDetails/id_random/
# codigo de permissão necessário --> 402: Pode visualizar checklists
response = requests.get("http://127.0.0.1:9000/api/checklist/CheckListLimpezaPredialDetails/x0GPde9yaCde/", headers=headers)
response.json()

{'id': 2,
 'id_random': 'x0GPde9yaCde',
 'servico_agendado': 1,
 'descricao': 'descricao de teste',
 'foto_comprovacao': None,
 'status': 'Concluído'}

# Atualizar checklist

In [16]:
# as chaves body da atualização é exatamente igual o body da criacao
# atualiando a descrição para checklist teste
body = {
    'servico_agendado': 1, #vem de api/servicos/agendados/ListServicoLimpezaPredialmAgendadoAnotados/
    'descricao': "checklist teste", # descircao do checklist 
    'foto_comprovacao': "", #tipo file comporvando o checklist 
    'status': "Concluído", # opcoes Concluído ou Pendente
}

In [19]:
# essa rota recebe o id random do checklist como parametro de url, consultar em /api/checklist/ListCheckListLimpezaPredial
# atualizando o checklist criada anteriormente, obeserve o id_random
# codigo de permissão necessário --> 401: Pode editar checklists
response = requests.put("http://127.0.0.1:9000/api/checklist/CheckListLimpezaPredialUpdate/x0GPde9yaCde/", body, headers=headers)
response.json()

{'success': True,
 'message': 'CheckListLimpezaPredial atualizado com sucesso.',
 'data': {'id': 2,
  'id_random': 'x0GPde9yaCde',
  'servico_agendado': 1,
  'descricao': 'checklist teste',
  'foto_comprovacao': None,
  'status': 'Concluído'}}

# Alterar status catalogo chceklist

### O checklist pode ter o statsu alterado direto em api/checklist/CheckListLimpezaPredialUpdate, enviando o "status" pelo body

# Se deletar o checklist

In [20]:
# essa rota recebe o id random do checklist como parametro de url, consultar em api/checklist/ListCheckListLimpezaPredial/
# codigo de permissão necessário --> 403: Pode excluir checklists
response = requests.get("http://127.0.0.1:9000/api/checklist/IfDeleteCheckListLimpezaPredail/x0GPde9yaCde/", headers=headers)
response.json()

{'objeto_principal': 'Checklist: checklist teste (Concluído)',
 'resumo': {'CheckListLimpezaPredial': 1},
 'objetos_em_cascata': ['- Checklist: checklist teste (Concluído)']}

# Deleção do checkList

In [22]:
# essa rota recebe o id random do checklist como parametro de url, consultar em api/checklist/ListCheckListLimpezaPredial/
# codigo de permissão necessário --> 403: Pode excluir checklists
response = requests.delete("http://127.0.0.1:9000/api/checklist/DeleteCheckListLimpezaPredail/x0GPde9yaCde/", headers=headers)
response.json()

{'detail': "CheckListLimpezaPredial 'Checklist: checklist teste (Concluído)' foi deletado com sucesso."}

# Endpoints Auxiliares

# Checklist associado a um servico especifico

In [23]:
# essa rota recebe  id random de um servico como parametro, obtido em api/servicos/agendados/ListServicoLimpezaPredialAgendadoAnotados/
# codigo de permissão 402: Pode visualizar checklists
response = requests.get("http://127.0.0.1:9000/api/checklist/ListCheckListLimpezaPredialByServico/UWoNKAzEeZRs/", headers=headers)
response.json()

{'success': True,
 'message': 'CheckListLimpezaPredial listado com sucesso.',
 'data': [{'id': 1,
   'id_random': 'Vqif9zIpzUqA',
   'servico_agendado': 1,
   'descricao': 'sub teste',
   'foto_comprovacao': None,
   'status': 'Pendente'}]}